In [1]:
import numpy as np

from helpers import pre_processing_wlan_utils as preprocess_utils
from helpers import classifier_wlan_spectral_utils as classifier_utils
from helpers import tr_models as tr_models

1 Physical GPUs, 1 Logical GPUs


In [2]:
import importlib
importlib.reload(tr_models)
importlib.reload(classifier_utils)
importlib.reload(preprocess_utils)

1 Physical GPUs, 1 Logical GPUs


<module 'helpers.pre_processing_wlan_utils' from 'C:\\Users\\migue\\development\\tc_spectrum\\code\\python\\helpers\\pre_processing_wlan_utils.py'>

In [3]:
#Set which task you want to solve. This value will be used to select the right labels for the loaded dataset.
#The three task can be 'phy', 'frames', 'app-type', or, 'app'
task = 'app'
label = preprocess_utils.label_index[task]
num_classes = preprocess_utils.num_classes[task]
labels_string = preprocess_utils.labels_string[task]
print("Label id: ", label)
print("Num classes in that label: ", num_classes)
print("Labels: ", labels_string)

Label id:  3
Num classes in that label:  8
Labels:  ['spotify', 'tunein', 'gpodcast', 'youtube', 'netflix', 'twitch', 'no-app', 'unknown']


In [4]:
#This dataset does not contain the unknown label class (L2 frames that were generated with an unknown app), so lets remove that label.
if (task == 'app') or (task == 'app-type'):
    num_classes = num_classes-1
    labels_string = labels_string[0:num_classes]
    print("Label id: ", label)
    print("Num classes in that label: ", num_classes)
    print("Labels: ", labels_string)

Label id:  3
Num classes in that label:  7
Labels:  ['spotify', 'tunein', 'gpodcast', 'youtube', 'netflix', 'twitch', 'no-app']


In [5]:
#Set lenght of the sequences
seq_length = 3000

#Set type of padding. 
padding = 'post'

#Set path to dataset folder. All the files from the dataset can be downloaded from https://zenodo.org/record/5208201
dataset_folder = '../../dataset/waveforms/'

#Set name of dataset file. In this case we are using once of the balanced dataset (filename_balanced.mat)
dataset_filename = 'waveforms_2G_n_SNR_'+task+'_balanced.mat'

In [6]:
#Get X and Y without padding/truncation nor scaling. The function get_raw_xy_spectrum returns the L1 packets (IQ samples) and all the labels associated to them. 
Xraw, Yraw = classifier_utils.get_raw_xy_spectrum(dataset_folder,dataset_filename)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 71603/71603 [01:21<00:00, 879.71it/s]


In [7]:
#Let's pad/truncate the L1 packets to a given lenght.
print("Padding/Truncating sequence to a length of ",str(seq_length))
X = classifier_utils.pad_or_trunc_x_and_scale(Xraw, seq_length, padding, scale=False)

Padding/Truncating sequence to a length of  3000


100%|█████████████████████████████████████████████████████████████████████████████████████████| 71603/71603 [00:34<00:00, 2102.42it/s]


In [8]:
#Lets select the labels of the classification task and generate the one-shot labels 
print("Generate one-shot labels")
Y = classifier_utils.get_one_hot_labels(Yraw, num_classes, label)

Generate one-shot labels
Labels created


In [9]:
#Set a seed for pseudo random generator
seed = 42
print("Performing data splitting")
X_train, X_val, X_test, Y_train, Y_val, Y_test = classifier_utils.get_xy_4_training(X,Y,seed)
print(X_train.shape,X_val.shape, X_test.shape)

Performing data splitting
Starting first partitioning
First partitioning done. Starting partitioning 2
Final partitioning done
(57282, 2, 3000) (7160, 2, 3000) (7161, 2, 3000)


In [10]:
model_type = 'CNN'

In [12]:
print('Starting data preparation and training for model ', model_type)
X_train, X_val, X_test = classifier_utils.reshape_for_model(model_type, X_train, X_val, X_test)
print(X_train.shape,X_val.shape, X_test.shape)

Starting data preparation and training for model  CNN
Re-shaping for CNN
(57282, 2, 3000, 1) (7160, 2, 3000, 1) (7161, 2, 3000, 1)


In [ ]:
result, model = classifier_utils.create_and_train_model(model_type, task, seq_length, num_classes, "text_now_", X_train, Y_train, X_val, Y_val, X_test, Y_test)

Creating the model
Model type CNN
Parameters for CNN model
dropout:  0.1
kernel size:  32
Number of Filters:  32
Model created
Model starts training
Model will be trained with the following configuration: 
Optimizer:  adam
Batch size:  64
Max number of epochs:  200
Model will be saved at  ./
Epoch 1/200
896/896 [==============================] - ETA: 5:40 - loss: 2.3646 - accuracy: 0.16 - ETA: 5:40 - loss: 2.3356 - accuracy: 0.17 - ETA: 5:39 - loss: 2.3108 - accuracy: 0.18 - ETA: 5:39 - loss: 2.2942 - accuracy: 0.18 - ETA: 5:39 - loss: 2.2861 - accuracy: 0.18 - ETA: 5:38 - loss: 2.2572 - accuracy: 0.19 - ETA: 5:38 - loss: 2.2401 - accuracy: 0.20 - ETA: 5:38 - loss: 2.2312 - accuracy: 0.20 - ETA: 5:38 - loss: 2.2344 - accuracy: 0.20 - ETA: 5:37 - loss: 2.2150 - accuracy: 0.20 - ETA: 5:36 - loss: 2.2185 - accuracy: 0.20 - ETA: 5:36 - loss: 2.2045 - accuracy: 0.20 - ETA: 5:36 - loss: 2.1965 - accuracy: 0.20 - ETA: 5:36 - loss: 2.1763 - accuracy: 0.21 - ETA: 5:35 - loss: 2.1725 - accuracy: